In [1]:
import os
import scipy.io
import numpy as np
from concurrent.futures import ThreadPoolExecutor, as_completed

# Function to process each file within a clip
def process_file(clip, file, input_dir, output_dir):
    file_path = os.path.join(input_dir, file)
    mat = scipy.io.loadmat(file_path)
    
    arrayREA = np.mean(mat['arrDREA'], axis=0)
    idx = file.split("_")[-1].split(".")[0]
    
    output_file_path = os.path.join(output_dir, f"cube_{idx}.npy")
    np.save(output_file_path, arrayREA)
    return f"{clip} {file} processed successfully"

# Function to handle the processing of a single clip, parallelizing file operations
def process_clip(clip):
    output_dir = os.path.join(output_base_dir, clip, "radar_polar_cube")
    input_dir = os.path.join(input_base_dir, clip, "radar_tesseract")

    os.makedirs(output_dir, exist_ok=True)
    file_names = sorted(os.listdir(input_dir))

    with ThreadPoolExecutor(max_workers=36) as executor:
        # Submitting file processing tasks for the current clip
        future_to_file = {executor.submit(process_file, clip, file, input_dir, output_dir): file for file in file_names}
        
        # As each file processing completes, you can add logging or error handling here
        for future in as_completed(future_to_file):
            file = future_to_file[future]
            try:
                result = future.result()
            except Exception as exc:
                print(f'File {file} in clip {clip} generated an exception: {exc}')
            else:
                print(result)

input_base_dir = "/mnt/Kradar/K-Radar/"
output_base_dir = "/mnt/12T-Data/Kradar/"
clips = sorted(os.listdir(input_base_dir), key=lambda x: int(x))

# Processing each clip, but within each clip, files are processed in parallel
for clip in clips:
    if clip == '1':
        continue
    process_clip(clip)

2 tesseract_00003.mat processed successfully
2 tesseract_00001.mat processed successfully
2 tesseract_00002.mat processed successfully
2 tesseract_00004.mat processed successfully
2 tesseract_00005.mat processed successfully
2 tesseract_00006.mat processed successfully
2 tesseract_00008.mat processed successfully
2 tesseract_00007.mat processed successfully
2 tesseract_00009.mat processed successfully
2 tesseract_00010.mat processed successfully
2 tesseract_00012.mat processed successfully
2 tesseract_00011.mat processed successfully


In [1]:
import numpy as np
import torch
data = np.load('/mnt/18T-Data/kradar/2/radar_polar_EAsparse/cube_00001.npy')

In [2]:
cube = torch.tensor(data)

In [14]:
cube_flat = cube.reshape(cube.shape[0], -1)
k = 100
# Use 'torch.topk' to get the indices of the top k elements
top_k_values, top_k_idx = torch.topk(cube_flat, k, dim=1)
# Create a mask with the same shape as cube_flat
mask_flat = torch.zeros_like(cube_flat, dtype=torch.bool)
mask_flat[torch.arange(cube_flat.shape[0])[:, None], top_k_idx] = True

# Calculate the indices for the range, elevation, and azimuth
range_inds = torch.arange(cube.shape[0])[:, None]
elevation_inds = top_k_idx // cube.shape[2]
azimuth_inds = top_k_idx % cube.shape[2]
range_ind = torch.repeat_interleave(range_inds, k).flatten()
elevation_ind = elevation_inds.flatten()
azimuth_ind = azimuth_inds.flatten()
# Extract the values at these indices from the original cube tensor
power_val = cube[range_ind, elevation_ind, azimuth_ind].unsqueeze(-1)
# thr_vale = arr_cube.quantile(quantile_rate)

/home/xiangyu/miniconda3/envs/OpenOccupancy/lib/python3.7/site-packages/ipykernel_launcher.py:11: UserWarning: __floordiv__ is deprecated, and its behavior will change in a future version of pytorch. It currently rounds toward 0 (like the 'trunc' function NOT 'floor'). This results in incorrect rounding for negative values. To keep the current behavior, use torch.div(a, b, rounding_mode='trunc'), or for actual floor division, use torch.div(a, b, rounding_mode='floor').
  # This is added back by InteractiveShellApp.init_path()


In [32]:
import torch
import torch.nn as nn

In [39]:
query = nn.Linear(64, 64, bias=False)

In [25]:
top_k_idx

tensor([[1113, 2182, 2289,  ..., 2504, 1002, 2606],
        [3048, 2941, 3155,  ...,  352, 1324, 3466],
        [3048, 2941,  139,  ..., 3688,  598,  492],
        ...,
        [3048, 3155, 2830,  ..., 3453, 3561, 3882],
        [3048, 3155, 2941,  ..., 1230, 3365,  357],
        [3048, 2941, 3155,  ..., 3603, 3257, 2489]])

In [24]:
top_k_values

tensor([[2.1709e+16, 2.1007e+16, 2.0417e+16,  ..., 6.4997e+15, 6.4883e+15,
         6.4773e+15],
        [2.8041e+13, 2.5031e+13, 2.3143e+13,  ..., 8.2490e+12, 8.2200e+12,
         8.2055e+12],
        [6.2471e+12, 5.5846e+12, 5.2108e+12,  ..., 2.1153e+12, 2.1036e+12,
         2.1007e+12],
        ...,
        [3.3195e+12, 3.2346e+12, 2.5564e+12,  ..., 9.7881e+11, 9.7858e+11,
         9.5820e+11],
        [6.8848e+12, 6.1828e+12, 5.6518e+12,  ..., 2.0597e+12, 2.0519e+12,
         2.0436e+12],
        [2.6107e+13, 2.2673e+13, 2.2203e+13,  ..., 8.2882e+12, 8.2065e+12,
         8.1713e+12]])

In [19]:
power_val

tensor([[2.1709e+16],
        [2.1007e+16],
        [2.0417e+16],
        ...,
        [8.2882e+12],
        [8.2065e+12],
        [8.1713e+12]])

In [27]:
cube[0,10,43]

tensor(2.1709e+16)

In [16]:
power_val.max()

tensor(2.1709e+16)

In [6]:
range_ind.shape

torch.Size([204800])

In [8]:
elevation_ind

tensor([10, 20, 21,  ...,  6, 20, 19])

In [9]:
azimuth_ind

tensor([43, 42, 42,  ..., 70, 43, 32])

In [7]:
range_ind

tensor([  0,   0,   0,  ..., 255, 255, 255])

In [5]:
power_val.shape

torch.Size([204800, 1])